# <b>Neural Network Model Design<b/>


## Import Cleaned Data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("data_cleaning/train_data.csv")

In [ ]:
df

In [76]:
df.shape


(90398, 66)

In [77]:
df["eval"] = df["eval"].clip(lower = -15, upper = 15)

In [78]:
df["eval"].min()

-15.0

In [79]:
X = df.drop(columns = "eval")

In [80]:
y = df[["eval"]]

In [81]:
y.head()

,eval
0,0.07
1,0.33
2,0.33
3,0.50
4,0.58


In [82]:
X_pred = X.iloc[[4]]
X_pred

,side_to_play,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64
4,-1.0,-5.0,-2.9,-3.0,0.0,-1.5,-3.0,-2.9,-5.0,-1.0,...,1.0,1.0,5.0,0.0,3.0,9.0,1.5,3.0,2.9,5.0


In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [84]:
X_train.shape

(63278, 65)

## Deep Learning Model 

In [57]:
from tensorflow.keras import Sequential, layers

In [117]:
# Model definition
model = Sequential()
model.add(layers.Dense(128, activation='relu', input_dim=65))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.summary()


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_80 (Dense)            (None, 128)               8448      
                                                                 
 dense_81 (Dense)            (None, 64)                8256      
                                                                 
 dense_82 (Dense)            (None, 64)                4160      
                                                                 
 dense_83 (Dense)            (None, 64)                4160      
                                                                 
 dense_84 (Dense)            (None, 64)                4160      
                                                                 
 dense_85 (Dense)            (None, 32)                2080      
                                                                 
 dense_86 (Dense)            (None, 16)              

In [118]:
model.compile(loss='mse', optimizer='adam')


In [123]:
model.fit(X_train, y_train, batch_size=70, epochs=10,verbose = 1)

Epoch 1/10
904/904 [==============================] - 3s 3ms/step - loss: 6.7743
Epoch 2/10
904/904 [==============================] - 2s 3ms/step - loss: 6.7381
Epoch 3/10
904/904 [==============================] - 4s 4ms/step - loss: 6.4411
Epoch 4/10
904/904 [==============================] - 4s 4ms/step - loss: 6.4836
Epoch 5/10
904/904 [==============================] - 3s 3ms/step - loss: 6.1900
Epoch 6/10
904/904 [==============================] - 3s 3ms/step - loss: 6.1119
Epoch 7/10
904/904 [==============================] - 2s 3ms/step - loss: 6.0537
Epoch 8/10
904/904 [==============================] - 3s 3ms/step - loss: 5.9978
Epoch 9/10
904/904 [==============================] - 2s 3ms/step - loss: 5.8999
Epoch 10/10
904/904 [==============================] - 3s 3ms/step - loss: 5.7131


In [124]:
model.evaluate(X_test, y_test, verbose = 1)

848/848 [==============================] - 2s 2ms/step - loss: 15.5335


15.533502578735352

## Exporting model

In [125]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


## Loading model

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

## Evaluating Loaded models

In [ ]:
loaded_model.compile(loss='RMSE', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X, y, verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))